In [1]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9da69d77904841e2eda35e695cb3278f54a4d4c31d92dc45427ad3367eee0abb
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [3]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

In [4]:
import pandas as pd

splits = {'train': 'final_headline_train_12000.csv', 'validation': 'final_headline_valid_1200.csv'}
train_df = pd.read_csv("hf://datasets/valurank/News_headlines/" + splits["train"])
val_df = pd.read_csv("hf://datasets/valurank/News_headlines/" + splits["validation"])

In [5]:
train_df.head()

,article,headline
0,The logo of cryptocurrency exchange Binance di...,Binance pauses bitcoin withdrawals due to a 's...
1,"Police officers, some in riot gear, guard a gr...",White nationalist group members face riot-plan...
2,A woman walks past a row of cash machines outs...,"Lloyds to give staff 1,000 pounds to ease cost..."
3,The Amazon logo is seen outside its JFK8 distr...,"Amazon offers to share data, boost rivals to d..."
4,An unexploded shell from a multiple rocket lau...,Both sides using heavier weapons in war in Ukr...


In [6]:
# Data cleaning function
def clean_article(article):
    # Remove metadata such as dates
    return article.split(') ', 1)[-1] if ') ' in article else article

In [7]:
# Apply cleaning to datasets
train_df['article'] = train_df['article'].apply(clean_article)
train_df.head()

,article,headline
0,The logo of cryptocurrency exchange Binance di...,Binance pauses bitcoin withdrawals due to a 's...
1,- Thirty-one members of the white nationalist ...,White nationalist group members face riot-plan...
2,- Britain's biggest domestic bank Lloyds (LLOY...,"Lloyds to give staff 1,000 pounds to ease cost..."
3,- Amazon (AMZN.O) has offered to share marketp...,"Amazon offers to share data, boost rivals to d..."
4,- Finnish President Sauli Niinisto said on Mon...,Both sides using heavier weapons in war in Ukr...


In [8]:
val_df['article'] = val_df['article'].apply(clean_article)

In [9]:
# Define a custom dataset class
class HeadlineDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length, target=True):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.target = target

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        row = self.dataframe.iloc[index]
        input_text = row['article']
        inputs = self.tokenizer(input_text, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt")

        if self.target:
            target_text = row['headline']
            targets = self.tokenizer(target_text, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt")
            return {
                'input_ids': inputs['input_ids'].squeeze(),
                'attention_mask': inputs['attention_mask'].squeeze(),
                'labels': targets['input_ids'].squeeze(),
                'target_text': target_text
            }
        else:
            return {
                'input_ids': inputs['input_ids'].squeeze(),
                'attention_mask': inputs['attention_mask'].squeeze()
            }

In [10]:
# Initialize tokenizer and dataset
tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-mul")
max_length = 256

train_dataset = HeadlineDataset(train_df, tokenizer, max_length)
val_dataset = HeadlineDataset(val_df, tokenizer, max_length)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/790k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/707k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [11]:
# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [12]:
# Initialize the model
model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-mul")

pytorch_model.bin:   0%|          | 0.00/310M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [13]:
# Define training parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

In [14]:
# Training loop
epochs = 25
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        del input_ids, attention_mask, labels, outputs
        torch.cuda.empty_cache()

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

Epoch 1/25, Loss: 0.35446199774742126
Epoch 2/25, Loss: 0.3043990731239319
Epoch 3/25, Loss: 0.24290908873081207
Epoch 4/25, Loss: 0.2696774899959564
Epoch 5/25, Loss: 0.23874035477638245
Epoch 6/25, Loss: 0.24299481511116028
Epoch 7/25, Loss: 0.19712349772453308
Epoch 8/25, Loss: 0.17791205644607544
Epoch 9/25, Loss: 0.1784573793411255
Epoch 10/25, Loss: 0.1355719417333603
Epoch 11/25, Loss: 0.12507264316082
Epoch 12/25, Loss: 0.10070690512657166
Epoch 13/25, Loss: 0.12591181695461273
Epoch 14/25, Loss: 0.08432068675756454
Epoch 15/25, Loss: 0.10157807916402817
Epoch 16/25, Loss: 0.08516817539930344
Epoch 17/25, Loss: 0.08482304215431213
Epoch 18/25, Loss: 0.07842923700809479
Epoch 19/25, Loss: 0.06287413090467453
Epoch 20/25, Loss: 0.07693453133106232
Epoch 21/25, Loss: 0.06445100903511047
Epoch 22/25, Loss: 0.03744320198893547
Epoch 23/25, Loss: 0.05170376971364021
Epoch 24/25, Loss: 0.043235618621110916
Epoch 25/25, Loss: 0.03612819314002991


In [15]:
# Save the trained model
model.save_pretrained("headline_generator_model")
tokenizer.save_pretrained("headline_generator_model")

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[64109]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('headline_generator_model/tokenizer_config.json',
 'headline_generator_model/special_tokens_map.json',
 'headline_generator_model/vocab.json',
 'headline_generator_model/source.spm',
 'headline_generator_model/target.spm',
 'headline_generator_model/added_tokens.json')

In [16]:
# Evaluate on the development dataset
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
all_rougel_scores = []
all_rouge1_scores = []
all_bleu_scores = []

model.eval()
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_length, num_beams=5)
        decoded_preds = [tokenizer.decode(output, skip_special_tokens=True, clean_up_tokenization_spaces=True) for output in outputs]
        decoded_targets = batch['target_text']

        for pred, target in zip(decoded_preds, decoded_targets):
            rouge_scores = scorer.score(target, pred)
            all_rougel_scores.append(rouge_scores['rougeL'].fmeasure)
            all_rouge1_scores.append(rouge_scores['rouge1'].fmeasure)
            target_tokens = target.split()
            pred_tokens = pred.split()
            bleu_score = sentence_bleu([target_tokens], pred_tokens)
            all_bleu_scores.append(bleu_score)

# Calculate average scores
avg_rouge_l = sum(all_rougel_scores) / len(all_rougel_scores)
avg_rouge_1 = sum(all_rouge1_scores) / len(all_rouge1_scores)
avg_bleu = sum(all_bleu_scores) / len(all_bleu_scores)

print(f"Average ROUGE-L Score: {avg_rouge_l:.4f}")
print(f"Average ROUGE-1 Score: {avg_rouge_1:.4f}")
print(f"Average BLEU Score: {avg_bleu:.4f}")


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Average ROUGE-L Score: 0.7965
Average ROUGE-1 Score: 0.8057
Average BLEU Score: 0.7816


In [17]:
model.push_to_hub('headline-generator-opus-mt-en-mul')
tokenizer.push_to_hub('headline-generator-opus-mt-en-mul')

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/willhsp/headline-generator-opus-mt-en-mul/commit/6109d57d7eb0bf7ce92f35f10c047ac37137ec4e', commit_message='Upload tokenizer', commit_description='', oid='6109d57d7eb0bf7ce92f35f10c047ac37137ec4e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/willhsp/headline-generator-opus-mt-en-mul', endpoint='https://huggingface.co', repo_type='model', repo_id='willhsp/headline-generator-opus-mt-en-mul'), pr_revision=None, pr_num=None)